In [2]:
!pip install praw transformers nltk emoji cohere dotenv

In [5]:
!pip install streamlit pyngrok
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
%%writefile app.py
import praw, re, json, emoji, cohere
import streamlit as st
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
import nltk
import dotenv


#Reddit API
reddit = praw.Reddit(
    client_id="XYdZOfOt56Y_c6VgzY9Lhw",
    client_secret="6M3LpUJ4fWS_a9rjV5OgOjLRnBGBtg",
    user_agent="sentiment-analyzer by u/potential_problem"
)

# Cohere API
co = cohere.Client("HIPZcL01IEuowrHCCluAD6VEEWmlDmmDcpFCiARj")

def extract_comments_from_post(post_url):
    submission = reddit.submission(url=post_url)
    submission.comments.replace_more(limit=0)
    comments_data = []
    for comment in submission.comments.list():
        comments_data.append({
            "comment_id": comment.id,
            "author": str(comment.author),
            "body": comment.body,
            "score": comment.score,
            "created_utc": comment.created_utc
        })
    return comments_data

def extract_emojis(text):
    return ''.join(c for c in text if c in emoji.EMOJI_DATA)

def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    tokens = word_tokenize(text.lower())
    return " ".join([t for t in tokens if t not in stopwords.words("english")])

# Sentiment model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

label_map = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "NEUTRAL",
    "LABEL_2": "POSITIVE"
}

def add_sentiment_to_comments(comments):
    for c in comments:
        try:
            c["cleaned"] = clean_text(c["body"])
            result = sentiment_pipeline(c["cleaned"][:512])[0]
            c["sentiment"] = label_map.get(result["label"], "UNKNOWN")
            c["sentiment_score"] = round(result["score"], 4)
        except:
            c["sentiment"] = "ERROR"
            c["sentiment_score"] = 0.0
    return comments

def calculate_sentiment_percentages(comments):
    sentiment_counts = Counter([c["sentiment"] for c in comments])
    total = sum(sentiment_counts.values())
    return {k: round((v / total) * 100, 2) for k, v in sentiment_counts.items()}

def summarize_comments_with_cohere(comments, sentiment_type="POSITIVE"):
    text_block = " ".join([c["cleaned"] for c in comments if c["sentiment"] == sentiment_type])
    if len(text_block.strip()) < 250:
        # Return raw or a fallback message if too short
        return f"Not enough {sentiment_type.lower()} comments to summarize properly."

    response = co.summarize(
        text=text_block[:4000],  # limit to 4k chars max
        model="summarize-xlarge",
        length="medium",
        format="paragraph",
        temperature=0.3,
        extractiveness="medium"
    )
    return response.summary


st.title("📊 Reddit Post Sentiment Analyzer")

post_url = st.text_input("Enter Reddit Post URL:")
if st.button("Analyze") and post_url:
    with st.spinner("Extracting and analyzing comments..."):
        comments = extract_comments_from_post(post_url)
        comments = add_sentiment_to_comments(comments)

        with open("reddit_sentiment_output.json", "w", encoding="utf-8") as f:
          json.dump(comments, f, indent=2)
        percents = calculate_sentiment_percentages(comments)
        pos_summary = summarize_comments_with_cohere(comments, "POSITIVE")
        neg_summary = summarize_comments_with_cohere(comments, "NEGATIVE")

    st.subheader("Sentiment Breakdown:")
    st.json(percents)

    st.subheader("🟢 Positive Summary:")
    st.write(pos_summary)

    st.subheader("🔴 Negative Summary:")
    st.write(neg_summary)


Overwriting app.py


In [9]:
from pyngrok import ngrok
import os
import time

# Kill previous processes
!pkill streamlit
ngrok.set_auth_token("2xadYlBxzGDr8KGuiMXfYIiL56Z_6ZKQM72Yvn9iPRceBnkvY")
# Open tunnel
public_url = ngrok.connect(8501, "http")  # Correct call: (port, "http")

print(f"🌐 Public URL: {public_url}")

# Wait for ngrok to set up
time.sleep(3)

# Launch Streamlit
!streamlit run app.py &>/content/log.txt &


🌐 Public URL: NgrokTunnel: "https://cf7d-34-125-152-230.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
post_url="https://www.reddit.com/r/oneplus/comments/1kk7s4u/bought_op13_after_watching_100s_of_videos_3/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button"
analyze_reddit_post(post_url)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



📥 Extracting comments...
🔍 Analyzing sentiment...

📊 Sentiment Percentages:
- POSITIVE: 33.33%
- NEUTRAL: 55.25%
- NEGATIVE: 11.42%

🟢 Summary of Positive Comments:
The user is enjoying their new OnePlus 13. They have watched many videos and have decided to stick with this phone instead of waiting for the next flagship. They mention that the phone is well-balanced, has good battery life, and has a smooth operating system. They also appreciate the phone's design, cameras, and speakers. They also talk about the competition and their own preferences. They also mention that they have ordered an OP12 as well, and are also considering other Chinese phones like Oppo and Poco.

🔴 Summary of Negative Comments:
User is considering buying the OnePlus 13 and is looking for information regarding the phone's specs, repairability and general performance. They are coming from an OnePlus 6T and are concerned about battery life and video quality. User is also frustrated with their current phone, a Googl